In [22]:
import re

def clean_ticker(ticker: str) -> str:
    """ Clean ticker of not necessary symbols (2U, 1000, 10000, SWAP, -, etc.) """
    ticker = re.sub(r'\b\d+', '', ticker)
    ticker = re.sub(r'2[a-zA-Z-]', '', ticker)
    ticker = re.sub('SWAP', '', ticker)
    ticker = re.sub('-', '', ticker)
    return ticker
    
clean_ticker('1000LUNA2FUSDT')

'LUNAUSDT'

# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 30, 16, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:40<00:00, 11.04s/it]


# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
19,72.917159,-1.013523,126.859545,-192.047841,3930
20,72.759394,-1.002197,120.562765,-193.762992,3952
18,72.463523,-1.007197,103.524167,-189.904924,3907
12,73.363144,-1.285682,99.527348,-181.789432,3767
16,72.789659,-1.079470,98.470417,-185.805568,3852
17,72.662576,-1.058182,97.679015,-188.584621,3880
2,84.642614,-4.187197,94.892159,-165.708636,3566
11,73.167008,-1.418598,89.151136,-181.487121,3740
15,72.866326,-1.135568,87.171212,-184.730379,3834


# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)